<a href="https://colab.research.google.com/github/mansivyas26/perfusion_dataPipeline/blob/main/variables_diff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import ast
import warnings
warnings.filterwarnings('ignore')

In [ ]:
perfusion_data = pd.read_pickle("/content/drive/MyDrive/Steve_BEXT84_01-17-2024_13-56-14_all_data.pkl")

In [ ]:
perfusion_data_subset = perfusion_data.head(5)

In [ ]:
variable_dict['possible_variables'] = variable_dict['possible_variables'].apply(ast.literal_eval)

In [ ]:
perfusion_data_subset

In [ ]:
variable_dict = pd.read_csv("/content/drive/MyDrive/Pefusion_DataPipeline/variable_dict_v_1.csv")



In [ ]:
schema = pd.read_csv("/content/m12_full_join.csv")

In [ ]:
schema_subset= schema.head(10)

In [ ]:
schema.head(5)

,dialysis_compensate_scale_flux,gas_mixer_percent_co2_measured,gas_mixer_percent_o2_measured,sensorbox_drain_solenoid_1_flag,arduino_reset_relay,dialysis_pressure_valve_position,pulse_generator_amplitude,gas_mixer_flow_o2_measured,gas_mixer_o2_ramp_flag,gas_mixer_percent_n2_measured,...,brain_pressure_target,main_pump_efficiency,glucose_pump_rate_setpoint,heat_exchanger_temperature_setpoint,glucose_syringe_diameter,gas_mixer_flow_measured,run_start_flag,gas_mixer_o2_ramp_time,heat_exchanger_temperature_measured,gas_mixer_co2_setpoint
0,NaN,6.0,21.0,NaN,NaN,NaN,5.0,146.0,NaN,73.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN
1,NaN,6.0,21.0,NaN,NaN,NaN,5.0,146.0,NaN,73.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN
2,NaN,6.0,21.0,NaN,NaN,NaN,5.0,146.0,NaN,73.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN
3,NaN,6.0,21.0,NaN,NaN,NaN,5.0,146.0,NaN,73.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN
4,NaN,6.0,21.0,NaN,NaN,NaN,5.0,146.0,NaN,73.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN


In [ ]:
def check_runs_with_variable_dict_new(pkl_df, variable_dict, schema_subset):
    removed_columns = []  # List to keep track of REMOVED columns

    columns = pkl_df.columns.tolist()
    for column in columns:
        match_found = False
        for idx, row in variable_dict.iterrows():
            if column in row['possible_variables']:
                match_found = True
                if row['variable_status'] == "EXISTING" and column != row['standard_variable']:
                    print(f"Changing column {column} to {row['standard_variable']}")
                    pkl_df.rename(columns={column: row['standard_variable']}, inplace=True)
                    if column in schema_subset.columns:
                        schema_subset.rename(columns={column: row['standard_variable']}, inplace=True)
                elif row['variable_status'] == "NEW":
                    print(f"Column status changes to existing. New column to be added to schema: {row['standard_variable']}")
                    if row['standard_variable'] not in schema_subset.columns:
                        schema_subset[row['standard_variable']] = None
                    row['variable_status'] = "EXISTING"
                """if row['variable_status'] == "REMOVED":
                    print(f"Removed column identified: {column}")
                    removed_columns.append(column)
                break
        if not match_found:
            print(f"Unidentified variable found: {column}")
            # Optionally handle adding unidentified variables to schema

    # Now, filter out REMOVED columns from pkl_df before any operation like appending to schema_subset
    pkl_df = pkl_df.drop(columns=removed_columns, errors='ignore')

    common_columns = set(pkl_df.columns).intersection(set(schema_subset.columns))
    schema_subset_filtered = schema_subset[list(common_columns)]
    pkl_df_filtered = pkl_df[list(common_columns)]

# Now append pkl_df to schema_subset
    combined_df = pd.concat([schema_subset_filtered, pkl_df_filtered], ignore_index=True)

    # Proceed with your operations, such as appending pkl_df to schema_subset, knowing REMOVED columns are excluded
    # Example: schema_subset = pd.concat([schema_subset, pkl_df], ignore_index=True)"""


In [ ]:
for index, row in variable_dict.iterrows():
    row['possible_variables'].append(row['standard_variable'])

In [ ]:
extract_first_ten = lambda lst: lst[:10]
perfusion_data_cols_subset = extract_first_ten(perfusio_data_cols)

In [ ]:
variable_dict_subset = variable_dict.head(10)

In [ ]:

perfusio_data_cols =perfusion_data.columns

In [ ]:
print(check_runs_with_variable_dict_new(perfusion_data,variable_dict_v_2_20formatted,schema_subset))

Column status changes to existing. New column to be added to schema: brain_flow_target
Column status changes to existing. New column to be added to schema: raman_glucose_mg_dl_measured
Changing column heat_exchanger_temperature_setpoint to heat_exchanger_temperature_setpoint_measured
Column status changes to existing. New column to be added to schema: mettler_ph_act
Column status changes to existing. New column to be added to schema: mettler_co2_dli
Column status changes to existing. New column to be added to schema: pulse_generator_control_mode
Column status changes to existing. New column to be added to schema: mettler_ph_dli
Column status changes to existing. New column to be added to schema: organ_resistance_regulation_flag
Column status changes to existing. New column to be added to schema: resistance_regulation_flag
Column status changes to existing. New column to be added to schema: mettler_o2_g_l
Column status changes to existing. New column to be added to schema: glucose_contr

In [ ]:
schema_subset


,dialysis_compensate_scale_flux,gas_mixer_percent_co2_measured,gas_mixer_percent_o2_measured,sensorbox_drain_solenoid_1_flag,arduino_reset_relay,dialysis_pressure_valve_position,pulse_generator_amplitude,gas_mixer_flow_o2_measured,gas_mixer_o2_ramp_flag,gas_mixer_percent_n2_measured,...,mettler_co2_ttm,mettler_o2_dli,heat_exchanger_oscillation_amplitude,mettler_ph_mv,mettler_o2_percent_air,brain_pressure_range_target,heat_exchanger_oscillation_flag,brain_flow_range_calculated,resistance_line_valve_position,resistance_regulation_flag
0,NaN,6.0,21.0,NaN,NaN,NaN,5.0,146.0,NaN,73.0,...,None,None,None,None,None,None,None,None,None,None
1,NaN,6.0,21.0,NaN,NaN,NaN,5.0,146.0,NaN,73.0,...,None,None,None,None,None,None,None,None,None,None
2,NaN,6.0,21.0,NaN,NaN,NaN,5.0,146.0,NaN,73.0,...,None,None,None,None,None,None,None,None,None,None
3,NaN,6.0,21.0,NaN,NaN,NaN,5.0,146.0,NaN,73.0,...,None,None,None,None,None,None,None,None,None,None
4,NaN,6.0,21.0,NaN,NaN,NaN,5.0,146.0,NaN,73.0,...,None,None,None,None,None,None,None,None,None,None
5,NaN,6.0,21.0,NaN,NaN,NaN,5.0,146.0,NaN,73.0,...,None,None,None,None,None,None,None,None,None,None
6,NaN,6.0,21.0,NaN,NaN,NaN,5.0,146.0,NaN,73.0,...,None,None,None,None,None,None,None,None,None,None
7,NaN,6.0,21.0,NaN,NaN,NaN,5.0,146.0,NaN,73.0,...,None,None,None,None,None,None,None,None,None,None
8,NaN,6.0,21.0,NaN,NaN,NaN,5.0,146.0,NaN,73.0,...,None,None,None,None,None,None,None,None,None,None
9,NaN,6.0,21.0,NaN,NaN,NaN,5.0,146.0,NaN,73.0,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
for column in schema_subset.columns:
  for idx, row in variable_dict_v_2_20formatted.iterrows():
            if column in row['possible_variables']:
                print(column)
            else:
              continue

dialysis_compensate_scale_flux
gas_mixer_percent_co2_measured
gas_mixer_percent_o2_measured
sensorbox_drain_solenoid_1_flag
arduino_reset_relay
arduino_reset_relay
dialysis_pressure_valve_position
pulse_generator_amplitude
gas_mixer_flow_o2_measured
gas_mixer_o2_ramp_flag
gas_mixer_percent_n2_measured
sensorbox_drain_solenoid_2_flag
gas_mixer_flow_setpoint
heat_exchanger_temperature_setpoint_measured
perfusion_time_seconds
dialysis_waste_scale_flux
dialysis_autoregulation_flag
perfusion_id
oxygenator_purge_solenoid_flag
dialysis_flux_rate_setpoint
air_supply_pressure
dialysis_compensate_scale_weight
heat_exchanger_status_flag
glucose_control_mode
glucose_control_mode
glucose_control_mode
probe_o2_temperature
gas_mixer_status_flag
viscosity_measured
brain_scale_weight
perfusion_date
probe_ph_temperature
dialysis_pre_pressure_setpoint
pulse_generator_autoregulation_flag
email_status_flag
glucose_syringe_direction
probe_partial_pressure_o2
glucose_syringe_pump_volume_dispensed_measured
pu